In [2]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread  


#### leer DBF simpe

In [36]:
class MyFieldParser(dbfread.FieldParser):

    # Sobrescribir el método parseN
    def parseN(self, field, data):
        # Si el campo es "importe", intentar convertir el valor a cadena
        if field.name == "importe":
            try:
                return str(data)
            # Si hay un error, devolver el valor original
            except ValueError:
                return data
        # Si el campo no es "importe", devolver el valor original
        else:
            return data
        
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            dbf =  dbfread.DBF( ruta_archivo, load=True, parserclass=MyFieldParser)
            indicadores = pd.DataFrame(dbf)
            #indicadores = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/INDICADORESV52.DBF' mode='r' encoding='cp1252'>


In [37]:
indicadores['IT_ITOT'].head(30)

0     b'      58000.00000000'
1     b'     148264.34000000'
2     b'      74232.00000000'
3     b'     142093.00000000'
4     b'       7339.00000000'
5     b'     216000.00000000'
6     b'      63525.00000000'
7     b'       2700.00000000'
8     b'     359975.00000000'
9     b'    2466225.50000000'
10    b'       7239.00000000'
11    b'       6990.00000000'
12    b'      55851.60000000'
13    b'       1329.00000000'
14    b'       2063.00000000'
15    b'       6059.00000000'
16    b'       6059.00000000'
17    b'      11434.23000000'
18    b'        457.37000000'
19    b'      10500.00000000'
20    b'      20641.00000000'
21    b'       6633.00000000'
22    b'    1095680.00000000'
23    b'    2048000.00000000'
24    b'      28183.00000000'
25    b'     388500.00000000'
26    b'      75000.00000000'
27    b'    -523100.00000000'
28    b'     523100.00000000'
29    b'     150460.28000000'
Name: IT_ITOT, dtype: object

### modelado

In [38]:
diccionario = {
'CONSULTA':[420301,
420132,
420125,
420134,
420126,
420127,
420130,
420121,
420117,
420129,
420110,
420104,
420145,
420105,
420102,
420109,
420128,
420108,
420101,
420103,
1,
420120,
3,
420133,
420107,
420112,
420143,
420135,
420106,
101,
420303,
420144,
420141]}

# creo la funcion para iterar el diciccionario

def aplicar_categoria(codigo):
    for categoria, codigos in diccionario.items():
        if codigo in codigos:
            return categoria
    return "practica"


# Crear campo pais
indicadores['PAIS'] = 'ARGENTINA' + ',' + indicadores['PROV_AFI'] + ',' + indicadores['LOCAAFI']

            

### aplicar diccionario y crear nuevo campo practicas_consultas

In [42]:
indicadores['CONSULTAS_PRACTICAS']=indicadores['IT_COD'].apply(aplicar_categoria)
#print(indicadores[['IT_COD','CONSULTAS_PRACTICAS']])
#save_path = filedialog.askdirectory()
#test
indicadores2 = indicadores[[
"ORD_ID",
"ORD_NUMERO",
"ORD_PERI",
"EXP_ID",
"OS_NOMBRE",
"NOMBRE",
"SEXO",
"EDAD",
"LOCAAFI",
"ORD_FSOL",
"ORD_FEC",
"NOMSOL",
"ORD_ORI",
"ORI_NOM",
"LOCAORI",
"ORD_AREA",
"NOMENCLA",
"IT_COD",
"NOM_NOM",
"CONSULTAS_PRACTICAS",
"CANTIDAD",
"IT_ITOT",
"MOT_NOM",
"NOMGTS",
"TEXP_NOMBR",
"PLAN_NOMBR",
"MONODROGA",
"ACCION",
"PAIS",
"PROV_AFI",
"IT_OBS",
"TIPOORIGEN",
"AGENTE_GTS",
"COB_ESP",
"DOC_ID",
"FECHAALTA"
]]
indicadores2.columns= indicadores2.columns.str.lower()
indicadores2['it_itot'] = indicadores2['it_itot'].replace('b':'',   regex=True)
#indicadores2.to_excel(save_path + '\indicadoresv5.xlsx', engine='openpyxl' ,index = False )
#indicadores2.to_csv(save_path + '\indicadoresv5.csv' ,index = False )
print(indicadores2['it_itot'])

SyntaxError: invalid syntax (3703884481.py, line 44)

### SUBIR EL EXCEL CORREGIDO

In [5]:
#cargar primero el excel o csv
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            indicadores3 = pd.read_excel(ruta_archivo)
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(indicadores)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/indicadoresv5.xlsx' mode='r' encoding='cp1252'>
       ORD_ID ORD_NUMERO  EXP_ID   OS_NOMBRE                  NOMBRE SEXO  \
0       25549  Amb-33444    8032  OSOETSYLRA    INEXISTENTE,AFILIADO    M   
1       25550  Amb-33445    8033  OSOETSYLRA    INEXISTENTE,AFILIADO    M   
2       25639  Amb-33536    8037     VISITAR    GONZALEZ,MARíA BELEN    F   
3       25641  Amb-33562    8052  OSOETSYLRA    PEON,RENZO VALENTINO    M   
4       25642  Amb-33569    8053  OSOETSYLRA        SOSA,AXEL HERNAN    M   
...       ...        ...     ...         ...                     ...  ...   
27148   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27149   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27150   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27151   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27152   42333      43593   11950     VISITAR  GIMENEZ,L

In [ ]:
indicadores3.style.format({'it_tot': '{:,.2f}'})

In [ ]:
indicadores['IT_ITOT']

### GUARDAR EL DATAFRAME EN UNA HOJA DE GSHEET

In [5]:
import pygsheets
import pandas as pd

path='C:\\Users\zickd\\OneDrive\Documentos\\repos_git\\signomedico-398902-d596060414cb.json'
gc=pygsheets.authorize(service_account_file=path)
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/10IKRk1MISpvgmRxH1pj_63Kt8PMEEJGvJp8I6unsa3c/edit?usp=drive_link')
wk1=sh[0] # first worksheet 
#wk1.clear() # to remove all data from sheet
wk1.set_dataframe(indicadores2,(1,1), extend=True, copy_head=True)

In [12]:
print(dbfread.__version__)



2.0.7
